# Manipuler les `DataArray`

## Introduction

Les `DataArray` (`DataArrayInt` et `DataArrayDouble`) sont utilisés dans
MEDCoupling pour stocker des valeurs sous forme de tableaux contigus en mémoire.
Les valeurs sont groupées par tuples, et chaque tuple a le même nombre de
composantes. Ils sont à la base de beaucoup de traitements réalisés dans
MEDCoupling. Il est donc important de bien savoir les manipuler.

Les `DataArrayDouble` sont souvent utilisés pour la manipulation directe des
valeurs d'un champ, comme on le verra plus tard. 

Les `DataArrayInt`, eux sont utilisés pour les manipulations des identifiants de
cellules et/ou de points.

## Résumé de l'exercice

L'Objectif de cet exercice est de se familiariser avec la manipulation des `DataArray`.

L'existe consiste à créer un maillage contenant 4 carrés adjacents.

<img src="four_squares.svg" style="width:300px;">

On commence par **créer un `DataArray`** (tableau), contenant les coordonnées
des nœuds d'un carré de côté $1$, et centré en $(0, 0)$. La première composante
du tableau de coordonnées s’appelle `X` avec l'unité `m` (mètre) et la 2ème
composante s'appellera `Y` (même unité).

On construit les coordonnées des nœuds des quatres carrés que l'on souhaite
obtenir en **dupliquant les coordonnées** des nœuds du carré original, puis en
leur appliquant une **translation**.

Il est ensuite nécessaire de **fusionner les nœuds présent en double**, issus
des multiples duplications.

Les notions abordées dans cet exercice sont :

- Créer une instance de `DataArrayDouble`
- Afficher une instance de `DataArrayDouble` et invoquer la méthode
  `getValue()`, pour la convertir en liste 
- Utiliser les notations de type "slice" `da[:,:]`
- Apprendre la renumérotation (convention `old-2-new`)
- Invoquer des services tels que `findCommonTuples()`


### Import de MEDCoupling 

Il est nécessaire d'importer le module Python `medcoupling`, pour pouvoir utiliser ses fonctionnalités.
On utilise l'alias `mc`, de manière à ne pas devoir réécrire `medcoupling.` à chaque appel de fonction.

Toutes les méthodes statiques du module commencent par une majuscule. Avec ces imports, sont disponibles:

- toutes les classes de MEDCoupling
- tous les énumérations (par exemple, les types de cellules standard: `mc.ON_CELLS`, `mc.ON_NODES`, `mc.ONE_TIME`...)
- toutes les méthodes statiques

Le module natif de Python `math` est également importé, pour réaliser des manipulations trigonométriques. 

In [ ]:
import medcoupling as mc
import math

### Créer une instance de `DataArrayDouble`, contenant 4 tuples

Le but ici est de créer une instance de `DataArrayDouble`, qui servira à stocker
les coordonnées des points aux angles d'un seul carré.

Il existe différentes manière de construire un `DataArray` :

In [ ]:
data_arr = mc.DataArrayDouble(4, 2)

Ici, les arguments `4` et `2` correspondent aux **dimensions** du `DataArray`.

Le premier argument `4`, indique le nombre de **lignes** du tableau. Cela
correspond, ici, au nombre de points servant à définir un carré. Il y aurait,
par exemple, fallu mettre `6`, pour stocker les points aux angles d'un hexagone.

Le second argument `2` indique le nombre de **colonnes**. Cela correspond, ici,
au nombre de coordonnées de chaque point.  Comme, dans cet exercice, on
travaille dans un repère 2D, chaque point possède bien deux coordonnées,
relatives aux axes $x$ et $y$.

On peut également construire un `DataArray` d'abord, et préciser ses dimensions dans un **second temps**.

In [ ]:
data_arr = mc.DataArrayDouble()
data_arr.alloc(4, 2)
print(data_arr)

On peut également construire un `DataArray` de **12 lignes**, puis le réorganiser en **2 colonnes**.

In [ ]:
data_arr = mc.DataArrayDouble(12)
data_arr.rearrange(2)
print(data_arr)

### Remarque

Pour les trois approches décrites ci-dessus, il est important de remarquer que les valeurs du tableau **ne sont pas initialisées**. Elles contiennent des valeurs qui n'ont aucun sens, dépendant de l'état de la mémoire de l'ordinateur.

Enfin, On peut construire un `DataArray` directement à partir d'une `list`
Python. Par défaut le tableau n'a **qu'une seule composante**.  Il faut donc
ensuite le réorganiser en **2 colonnes**.


In [ ]:
data_arr = mc.DataArrayDouble([0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0])
data_arr.rearrange(2)
print(data_arr)

Toutes ces approches sont différentes manière de construire un tableau `d` contenant 12 valeurs, groupées en 4 tuples contenant chacun 2 composantes. Chaque tuple représente les coordonnées 2D d'un point.

### Affichage d'un tableau

Jusqu'ici, le contenu d'un `DataArray` a été affiché à l'aide de la fonction
`print()`, prenant directement le tableau en argument. Cette approche affiche de
nombreuses méta-informations à propos l'objet.  Il est également possible de
n'afficher que les valeurs brutes du tableau, à l'aide de la syntaxe suivante :


In [ ]:
print(data_arr.getValues())

### Assignation de valeurs

L'assignation de valeurs à un `DataArray` est très proche de la syntaxe utilisée pour les `ndarray` de la librairie `numpy`. On peut notamment utiliser le `slicing`, à l'aide du symbole `:`.

In [ ]:
data_arr[1:4, 0] = 3.0

On peut, par exemple, directement assigner des valeurs à la deuxième colonne du tableau à l'aide d'une liste.

In [ ]:
data_arr[:, 1] = [1.0, 2.0, 3.0, 4.0]

On assigne des valeurs à `d`, de manière à ce qu'il contienne coordonnées cartésiennes des nœuds d'un carré centré en `(0, 0)`, de côté `1`.

In [ ]:
data_arr[:, 0] = [-0.5, 0.5, 0.5, -0.5]
data_arr[:, 1] = [-0.5, -0.5, 0.5, 0.5]
print(data_arr)

**Remarque :** Si nécessaire, il est possible de travailler en coordonnées
polaires, et de réaliser des conversions entre coordonnées polaires et
cartésiennes.


In [ ]:
data_arr_pol = data_arr.fromPolarToCart()
print(data_arr_pol)

data_arr_cart = data_arr_pol.fromCartToPolar()
print(data_arr_cart)

On peut assigner des **noms aux différentes composantes (colonnes) d'un `DataArray`**. Cela permet de donner des indications claire sur la manière dont il faut interpréter les données du tableau. On peut notamment renseigner le nom des coordonnées (cartésiennes, ici), et les unités (le mètre, ici).

In [ ]:
data_arr.setInfoOnComponents(mc.svec(["X [m]", "Y [m]"]))
print(data_arr)

Dans le cadre de cet exercice, cela n'est pas indispensable. Cependant, d'autres fonctions plus avancées **nécessitent cette information**.

Les quatres nœuds du carré dont on a construit les coordonnées appartiennent au cercle de centre $(0, 0)$ et de rayon $\frac{\sqrt{2}}{2}$. On peut vérifier que la norme (`magnitude()`) de chaque tuple de `d` est bien égale à $\frac{\sqrt{2}}{2}$, avec une tolérance de `1.e-12`, grâce à la méthode `isUniform()`.


In [ ]:
print(data_arr.magnitude())

tolerance = 1.0e-12

print("Is 'd' uniform ?", data_arr.magnitude().isUniform(math.sqrt(2) / 2.0, tolerance))

### Duplication et agrégation

On construit maintenant la liste `translationToPerform`, qui contient une liste de vecteurs de taille 2. Cette liste de taille 4 (4 carrés), contient les différentes translations à opérer pour obtenir les coordonnées des nœuds des quatre carrés que l'on souhaite construire.

In [ ]:
translations = [
    [0.5, 0.5],
    [0.5, 1.5],
    [1.5, 0.5],
    [1.5, 1.5],
]

On crée les quatre tableaux, contenant les coordonnées translaté des nœuds du carré original.

In [ ]:
# list of 'DataArrayDouble'
squares = []
for translation in translations:
    # Adding a vector to a set of coordinates does a translation. translation
    # could have been a DataArrayDouble too.
    squares.append(data_arr + translation)
print(squares)

## Agrégation de tableaux

A partir de cette liste d'instances de `DataArrayDouble` , on construit un
unique `DataArrayDouble`, résultat de l'agrégation des instances, les unes à la
suite des autres. On utilise, pour cela, la méthode `Aggregate`.

In [ ]:
squares_da = mc.DataArrayDouble.Aggregate(squares)
print(squares_da)

On a ainsi construit un `DataArrayDouble`, contenant les coordonnées des nœuds
des quatres carrés que l'on souhaite construire. Une remarque importante est que
l'ordre d’agrégation des `DataArray` est préservé.

**Remarques:**

- Il en est de même pour l’agrégation de maillages et de champs, de
  manière à faciliter l'accès et le repérage des données. C'est par exemple une
  différence essentielle avec le modèle `MEDFichier`, comme on le verra plus tard.
- Il existe également la méthode `mc.DataArrayDouble.Meld(arr)`, permettant d'agréger deux `DataArray` composante par composante, c'est-à-dire de concaténer des
  tableaux colonne par colonne, plutôt que ligne par ligne.


### Trouver les tuples égaux

Dans le `DataArray` construit précédemment, certains tuples sont identiques. Il
est nécessaire de détecter les tuples égaux, pour éviter de créer des nœuds
doublons.  Pour trouver les tuples égaux, on utilise `findCommonTuples()`. Pour comparer deux tuples de nombre flottants entre eux, cette fonction prend en argument une tolérance absolue. 


**Remarque :**
On peut utiliser `help(mc.DataArrayDouble.findCommonTuples)` pour obtenir des
informations sur la manière d'utiliser cette fonction. l'interface. 


In [ ]:
print("number of tuples: ", squares_da.getNumberOfTuples())

# help(mc.DataArrayDouble.findCommonTuples)

absolute_tolerance = 1.0e-12

common_tuples, common_tuples_indices = squares_da.findCommonTuples(absolute_tolerance)
print("common_tuples:\n", common_tuples.getValues())
print("common_tuples_indices:\n", common_tuples_indices.getValues())

La fonction `findCommonTuples` renvoie deux `DataArrayInt`. La première contient
les groupes d'indices de nœuds communs, concaténés entre eux. La seconde liste
indique la position du début de chacun des groupes, dans la première liste.

Ce format de retour en deux `DataArrayInt` est assez courant dans MEDCoupling,
pour des raisons de performance. Il s'appelle l'**indirect indexing**. Ce format
est particulièrement utilisé pour les fonctions manipulant des maillages
non-structurés.

**Attention, le dernier élément du deuxième tableau pointe en dehors du tableau du premier tableau**.
Ce dernier index est toujours présent et permet de s'assurer que des traitements
tels que les slices présentés juste après, sont toujours valables, sans avoir
besoin de particulariser le dernier groupe.

Voici un schéma illustrant le fonctionnement de l'indirect indexing, de manière générale :

<img src="IndirectIndex.jpg" style="width:700px;">

Le nombre de groupe est donc égal à la taille du deuxième tableau, celui contenant les indices, moins un.

In [ ]:
c = common_tuples
ci = list(common_tuples_indices)
for i, (j, k) in enumerate(zip(ci[:-1], ci[1:])):
    print("group", i, ": ", c[int(j) : int(k)].getValues())

Par exemple, on peut vérifier que les tuples `1` et `8`, ceux du groupe `0`, sont bien égaux:

In [ ]:
grp_index = 0
grp = c[int(ci[grp_index]) : int(ci[grp_index + 1])]
for i in grp:
    print(squares_da[i].getValues())

Dans un indirect indexing, on peut calculer la taille de chaque
groupe, sans avoir à manipuler directement le tableau d'indices (i.e. le deuxième
tableau). On utilise, pour cela, la fonction `DataArrayInt.deltaShiftIndex`.

In [ ]:
print(common_tuples_indices.deltaShiftIndex().getValues())

### Construire un tableau "old-2-new"

Grâce à la détection des nœuds doublons, effectuée précédemment, il est possible de créer un nouveau tableau dans lequel les doublons ont été éliminés.

Mathématiquement, fusionner des nœuds doublons en un seul nœud revient à
effectuer une surjection d'un espace de départ `X` (ici à 16 nœuds) vers un
espace d'arrivé `Y`, à 9 nœuds.

<img src="SurjectionDataArray.png" style="width:250px;">

Dans MEDCoupling, la manière de représenter cette surjection est de la
par un `DataArrayInt`, stockant des indices, avec la convention MEDCoupling
appelée **"old-to-new"**.

**Remarque :** La convention "old-to-new" est, par convention, celle à privilégier
également pour les bijections (e.g. une permutation).

Dans cette convention, chaque élément d'indice $i$ de ce tableau
contient le nouvel identifiant du tuple $i$ de `X`, dans l'ensemble d'arrivé `Y`.

Par exemple, `[0, 1, 2, 2]`, en convention "old-to-new", signifie que:
- `X[0]` correspond à `Y[0]`
- `X[1]` correspond à `Y[1]`
- `X[2]` correspond à `Y[2]`
- `X[3]` correspond à `Y[2]`

Nous allons construire ce tableau pour extraire un sous-ensemble des coordonnées
de départ, et ne conserver que les tuples uniques (non doublons). 

La méthode statique `DataArrayInt.ConvertIndexArrayToO2N()` permet de passer du
mode de stockage de cette surjection, représentée par un indirect indexing, au
format "old-to-new". Le suffix `O2N` de la fonction se lit "OldToNew". Cette
fonction renvoie également $Card(Y)$, c'est-à-dire le nombre de tuples après
élimination des doublons, ici.

In [ ]:
o2n, newNbOfTuples = mc.DataArrayInt.ConvertIndexArrayToO2N(
    len(squares_da), common_tuples, common_tuples_indices
)
print("old-to-new list: ", o2n.getValues())
print("new number of tuples: ", newNbOfTuples)

Dans le maillage final, on souhaite bien n'avoir que neuf nœuds (c.f. le schéma des quatre carrés, au début de l'exercice).

Nous pouvons maintenant construire le tableau de tuples uniques, à l'aide de
`o2n` et `newNbOfTuples`. On utilise, pour cela, la fonction
`DataArrayDouble.renumberAndReduce()`.

In [ ]:
squares_nodes_no_duplicate = squares_da.renumberAndReduce(o2n, newNbOfTuples)

Il est possible de translater tous les tuples d'un `DataArray` d'un seul coup,
en additionnant directement une liste de même dimension que les tuples du
tableau :

In [ ]:
squares_nodes_no_duplicate += [1.0, 1.0]

### Construire un maillage non-structuré

On peut, à présent, créer un maillage non-structuré 2D, en utilisant le tableau
construit pour définir les coordonnées des nœuds du maillage.


In [ ]:
dimension = 2

mesh = mc.MEDCouplingUMesh("FourSquares", dimension)
mesh.setCoords(squares_nodes_no_duplicate)

print("Mesh dimension is", mesh.getMeshDimension())
print("Spatial dimension is", mesh.getCoords().getNumberOfComponents())

Pour l'instant, le maillage créé ne contient qu'une liste de nœuds. Il ne
contient aucune cellule.

Pour créer des cellules dans ce maillage,, il faut créer un groupe d'indices de
nœuds d'une cellule à ajouter, puis utiliser la fonction `insertNextCell`. Ce
groupe d'indices de nœuds s'appelle la table de connectivité de la cellule. 

Premièrement, Il est nécessaire d'allouer de la mémoire pour les cellules à créer:


In [ ]:
mesh.allocateCells(4)

Enfin, grâce au tableau "old-to-new" construit précédemment, on peut créer la
table de connectivité de chaque cellule du maillage, i.e. des quatre carrés.


In [ ]:
print(o2n.getValues())
for i in range(4):
    # iter through o2n by chunks of 4 nodes at a time
    square_node_indices = o2n[4 * i : 4 * (i + 1)]
    mesh.insertNextCell(mc.NORM_POLYGON, square_node_indices.getValues())

On vérifie que le maillage ne contient pas d'anomalie.


In [ ]:
mesh.checkConsistencyLight()

C'est toujours une bonne idée d'appeler cette méthode après la construction d'un
maillage. Cela assure qu'il n'y a pas d'erreurs grossières, notamment dans la
table de connectivité du maillage.

Pour vérifier visuellement que le maillage est correct, on l'exporte dans un
fichier au format `.vtu`, que l'on peut visualiser dans `Paraview` ou le module
`ParaVIS` de la plateforme `Salome`.

In [ ]:
mesh.writeVTK(mesh.getName() + ".vtu")

**Note :** On exporte ici le maillage au format `.vtu` et non `.med`, car
MEDCoupling n'inclut pas, par défaut, la librairie `MED-file`, nécessaire à la
génération des `.med`.